In [39]:
import pickle
import numpy as np
import pandas as pd
from pprint import pprint
from time import time
import logging
import sys
import tensorflow as tf
from os import path
from os import listdir
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow import feature_column
#import tensorflow_hub as hub
from tensorflow.keras.callbacks import TensorBoard
from azureml.core import Dataset, Workspace
from tempfile import TemporaryDirectory
import tempfile
#import tensorflow_addons as tfa

state_gov_nm = 'MA'
year_to_process = '2020'
data_file_location = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/beefy/code/QuigleyData'
vector_size = [96]
cluster_size = [30]
#vector_size = [32, 64, 96, 128, 160]
#cluster_size = [10, 30, 50, 70, 100]

In [40]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [41]:
ws = Workspace.get(name="ml-quigley"
                   , subscription_id='8d8bcbfe-86b4-42bb-a470-a8cf97c98d69'
                   , resource_group='project-quigley')

local_ds_path = tempfile.mkdtemp()

#define the azureml dataset
leg_meta_ds = Dataset.get_by_name(ws,'leg_meta_ds')

In [42]:
_cmltv_event_df = path.join(data_file_location, state_gov_nm + 'cmltv_events.df')

for v in vector_size:
    for c in cluster_size:
        years = []
        chambers = []
        vectors = []
        leg_data_raw = pd.DataFrame()
        LegSponsorDF = pd.DataFrame()
        if 1==1:
        #with TemporaryDirectory() as local_ds_path:
            with leg_meta_ds.mount(local_ds_path):
                for f in listdir(local_ds_path):
                    if state_gov_nm in f:
                        _LegOverallData_CSV = path.join(local_ds_path, f)
                        leg_data_raw = leg_data_raw.append(pd.read_csv(_LegOverallData_CSV))
                        #print(leg_data['chamber'].dtype)
        
            leg_people_ds = Dataset.get_by_name(ws,'leg_people_ds')

            with leg_people_ds.mount(local_ds_path):
                for f in listdir(local_ds_path):
                    if 'LegPeopleData.csv' in f and state_gov_nm in f:
                        
                        _LegPeopleData_CSV = path.join(local_ds_path,f)
                        LegSponsorDF = LegSponsorDF.append(pd.read_csv(_LegPeopleData_CSV))

                    vec_temp = 0
                    if 'SessionBodyVector.dict' in f and state_gov_nm in f:
                        _SessionBodyVector = path.join(local_ds_path, f)
                        with open(_SessionBodyVector,'rb') as fl:
                            vec_temp = pickle.load(fl)
                            years.extend(vec_temp['year'])
                            chambers.extend(vec_temp['chamber'])
                            vectors.extend(vec_temp['partisan_vector'])

       
        #_legtext = path.join(data_file_location, state_gov_nm + str(v) + 'embedded_legdoc.df')
        _bill_clusters = path.join(data_file_location, state_gov_nm + str(v) + str(c) + 'bill_clusters.df')
        
        #with open(_legtext,'rb') as f:
        #    legtextembed = pickle.load(f)

        with open(_cmltv_event_df,'rb') as f:
            leg_event_accumulated = pickle.load(f)

        with open(_bill_clusters,'rb') as f:
            leg_doc_clusters = pickle.load(f)

        leg_data = leg_data_raw

        LegSponsorDF['sponsors'] = pd.Categorical(LegSponsorDF['sponsors'],categories=LegSponsorDF['sponsors'].unique())

        LegSponsorDF = pd.get_dummies(LegSponsorDF, columns=['sponsors'])
        LegSponsorDF = LegSponsorDF.groupby(['bill_id']).sum()

        #create the session vector dictionary
        SessionBodyVectors = {'year' : years, 'chamber' : chambers, 'partisan_vector' : vectors}
        SessionBodyMakeup = pd.DataFrame(SessionBodyVectors)
        SessionBodyMakeup['year'] = SessionBodyMakeup['year'].astype('int16')
        SessionBodyMakeup['chamber'] = SessionBodyMakeup['chamber'].astype('str')
        ##########################################################################################
        #Clean up and combine the dataframes
        leg_doc_clusters['bill_id'] = leg_doc_clusters['bill_id'].astype('int64')
        leg_data['bill_id'] = leg_data['bill_id'].astype('int64')
        leg_data['time_t'] = leg_data['time_t'].astype('int8')
        leg_event_accumulated['time_t'] = leg_event_accumulated['time_t'].astype('int8')
        leg_data = leg_data.drop(columns=['event', 'next_event', 'previous_event', 'Unnamed: 0', 'primary_id','title','committee_introduced'])
        #chamber                          object
        #bill_type                        object
        #proposed_chamber                 object
        #primary_name                     object
        #primary_party                    object
        leg_data['bill_type'] = leg_data['bill_type'].astype('str')
        leg_data['proposed_chamber'] = leg_data['proposed_chamber'].astype('str')
        leg_data['primary_name'] = leg_data['primary_name'].astype('str')
        leg_data['primary_party'] = leg_data['primary_party'].astype('str')
        leg_data['chamber'] = leg_data['chamber'].astype('str')
        leg_data['chamber'] = leg_data['chamber'].fillna('None')
        leg_data['chamber'] = leg_data['chamber'].replace(to_replace='A', value='H')
        print(len(leg_data[leg_data['chamber'] == 'None']))
        leg_data['primary_name'] = leg_data['primary_name'].fillna('None')
        leg_data['primary_party'] = leg_data['primary_party'].fillna('N')

        #adjust the number of sponsors, based on the max value found in the field (for future iterations, just divide by the number in that particular chamber)
        max_sponsors = leg_data['number_sponsors'].max()
        leg_data['num_sponsors_adj'] = (leg_data['number_sponsors']*1.00)/max_sponsors

        #filtering out all types of legislation except bills (no resolutions, constitutional ammendments, etc.)
        leg_data = leg_data[leg_data['bill_type']=='B']

        #print(len(leg_data), len(legtextembed))

        #leg_data = leg_data.join(legtextembed)
        print(len(leg_data))
        #print(len(leg_data[leg_data['year']==int(year_to_process)]))
        leg_data = pd.merge(leg_data, leg_doc_clusters, how='inner', on=['bill_id'])
        print(len(leg_data))
        #print(len(leg_data[leg_data['year']==int(year_to_process)]))
        leg_data = pd.merge(leg_data, leg_event_accumulated, how='inner', on=['bill_id','time_t'])
        print(len(leg_data))
        #print(len(leg_data[leg_data['year']==int(year_to_process)]))
        #print(leg_data['cmltv_event'].head)
        leg_data = pd.merge(leg_data, LegSponsorDF, how='inner', on=['bill_id'])
        print(len(leg_data))
        #print(len(leg_data[leg_data['year']==int(year_to_process)]))
        if state_gov_nm not in ['MA']: #some states don't have the data needed for a partisasn vector.  those states need manual creation of the vector files
            leg_data = pd.merge(leg_data, SessionBodyMakeup, how='inner', on=['year','chamber'])
        else:
            leg_data['partisan_vector'] = [.5] * len(leg_data)
        print(len(leg_data))
        #print(len(leg_data[leg_data['year']==int(year_to_process)]))
        try:
            leg_data = leg_data.drop(columns=['Unnamed: 0'])
        except:
            print('No additional Unnamed Column')
        _dataframe_export = path.join(data_file_location, state_gov_nm + str(v) + str(c) + '_dataset.df')
        
        
        with open(_dataframe_export, 'wb') as f:
            pickle.dump(leg_data, f)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0
437473
437473
437460
433578
433578
No additional Unnamed Column


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0
437473
437473
437460
433578
433578
No additional Unnamed Column


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

0
437473
437473
437460
433578
433578
No additional Unnamed Column


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0
437473
437473
437460
433578
433578
No additional Unnamed Column


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0
437473
437473
437460
433578
433578
No additional Unnamed Column


In [43]:
print(leg_data.dtypes)

bill_id                                                     int64
numeric_date                                              float64
time_t                                                       int8
sponsor_vec                                               float64
session                                                     int64
year                                                        int64
chamber                                                    object
bill_type                                                  object
proposed_chamber                                           object
primary_name                                               object
primary_party                                              object
number_sponsors                                             int64
month_introduced                                            int64
year_start                                                  int64
intro_to_end_of_session                                   float64
last_event